In [3]:
import importlib
from ucr_benchmark_template.modeling import train_drtp

importlib.reload(train_drtp)

from ucr_benchmark_template.modeling.train_drtp import (
    load_dataset, make_model, train, predict, evaluate, save_model
)

In [ ]:
import optuna
import numpy as np
import pandas as pd
import os

def objective(trial):
    datasets = [
        "ACSF1", "Adiac", "ArrowHead", "BME", "Beef", "BeetleFly", "BirdChicken", "CBF", "Car", "Chinatown",
        "ChlorineConcentration", "CinCECGTorso", "Coffee", "Computers","CricketX", "CricketY", "CricketZ", "Crop",
        "DiatomSizeReduction", "DistalPhalanxOutlineAgeGroup", "DistalPhalanxOutlineCorrect", "DistalPhalanxTW",
        "ECG200", "ECG5000", "ECGFiveDays", "EOGHorizontalSignal", "EOGVerticalSignal", "Earthquakes", "ElectricDevices",
        "EthanolLevel", "FaceAll", "FaceFour", "FacesUCR", "FiftyWords", "Fish", "FordA", "FordB", "FreezerRegularTrain",
        "FreezerSmallTrain", "GunPoint", "GunPointAgeSpan", "GunPointMaleVersusFemale", "GunPointOldVersusYoung", "Ham",
        "HandOutlines", "Haptics", "Herring", "HouseTwenty", "InlineSkate", "InsectEPGRegularTrain", "InsectEPGSmallTrain",
        "InsectWingbeatSound", "ItalyPowerDemand", "LargeKitchenAppliances", "Lightning2", "Lightning7", "Mallat", "Meat",
        "MedicalImages", "MiddlePhalanxOutlineAgeGroup", "MiddlePhalanxOutlineCorrect", "MiddlePhalanxTW",
        "MixedShapesRegularTrain", "MixedShapesSmallTrain", "MoteStrain", "NonInvasiveFetalECGThorax1",
        "NonInvasiveFetalECGThorax2", "OSULeaf", "OliveOil", "PhalangesOutlinesCorrect", "Phoneme", "PigAirwayPressure",
        "PigArtPressure", "PigCVP", "Plane", "PowerCons", "ProximalPhalanxOutlineAgeGroup", "ProximalPhalanxOutlineCorrect",
        "ProximalPhalanxTW", "RefrigerationDevices", "Rock", "ScreenType", "SemgHandGenderCh2", "SemgHandMovementCh2",
        "SemgHandSubjectCh2", "ShapeletSim", "ShapesAll", "SmallKitchenAppliances", "SmoothSubspace",
        "SonyAIBORobotSurface1", "SonyAIBORobotSurface2", "StarLightCurves", "Strawberry", "SwedishLeaf", "Symbols",
        "SyntheticControl", "ToeSegmentation1", "ToeSegmentation2", "Trace", "TwoLeadECG", "TwoPatterns", "UMD",
        "UWaveGestureLibraryAll", "UWaveGestureLibraryX", "UWaveGestureLibraryY", "UWaveGestureLibraryZ", "Wafer", "Wine",
        "WordSynonyms", "Worms", "WormsTwoClass", "Yoga"
    ]

    # --- Hyperparameters ---
    dropout = trial.suggest_categorical("dropout", [0.1, 0.2, 0.3])
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    k_size = trial.suggest_categorical("k_size", [3, 5])

    n_blocks = trial.suggest_int("n_blocks", 1, 3)
    channels = [1]
    for i in range(n_blocks):
        ch = trial.suggest_categorical(f"channels_block{i}", [8, 16, 32, 64, 128])
        channels.append(ch)

    n_layers = trial.suggest_int("n_layers", 2, 4)
    fc_layers = []
    for i in range(n_layers):
        l = trial.suggest_categorical(f"layer_size{i}", [128, 256, 512])
        fc_layers.append(l)

    batch = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 70, 120)


    fc_layers = [512, 256]
    channels: [1, 16, 32]
    
    # --- Evaluate ---
    accuracies = []
    for dataset in datasets:
        print(dataset)
        trainloader, testloader = load_dataset(dataset)
        model = make_model(dataset, channels, k_size, dropout, fc_layers)
        model, _ = train(model, trainloader, lr, epochs)
        y_true, y_pred = predict(model, testloader)
        acc = evaluate(y_true, y_pred)["accuracy"]
        accuracies.append(acc)

    avg_acc = np.mean(accuracies)

    # --- Save results to one CSV with all runs ---
    csv_path = "optuna_results_drtp.csv"

    # Create base table if doesn't exist
    if not os.path.exists(csv_path):
        df = pd.DataFrame({"Dataset": datasets + ["AVERAGE"]})
    else:
        df = pd.read_csv(csv_path)

    # Make trial name
    trial_name = f"Trial_{trial.number+1}"

    # Add this run’s results
    run_data = accuracies + [avg_acc]
    df[trial_name] = run_data

    # Also add hyperparameters as a separate column name pattern
    hp_str = (
        f"lr={lr:.1e}, k={k_size}, dropout={dropout} "
        f"blocks={n_blocks}, ch={channels}, layers={n_layers}, "
        f"fc={fc_layers}, batch={batch}, epochs={epochs}"
    )

    df.loc[df["Dataset"] == "AVERAGE", f"{trial_name}_params"] = hp_str

    df.to_csv(csv_path, index=False)

    return avg_acc


# --- Run Study ---
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)


[I 2025-10-07 14:20:35,048] A new study created in memory with name: no-name-84044f73-f9f1-4db9-8830-d87aaf4608b0


ACSF1
Adiac
ArrowHead
BME
Beef
BeetleFly
BirdChicken
CBF
Car
Chinatown
ChlorineConcentration
CinCECGTorso
Coffee
Computers
CricketX
CricketY
CricketZ
Crop
DiatomSizeReduction
DistalPhalanxOutlineAgeGroup
DistalPhalanxOutlineCorrect
DistalPhalanxTW
ECG200
ECG5000
ECGFiveDays
EOGHorizontalSignal
EOGVerticalSignal
Earthquakes
ElectricDevices
EthanolLevel
FaceAll
FaceFour
FacesUCR
FiftyWords
Fish
FordA
FordB
FreezerRegularTrain
FreezerSmallTrain
GunPoint
GunPointAgeSpan
GunPointMaleVersusFemale
GunPointOldVersusYoung
Ham
HandOutlines
Haptics
Herring
HouseTwenty
InlineSkate
InsectEPGRegularTrain
InsectEPGSmallTrain
InsectWingbeatSound
ItalyPowerDemand
LargeKitchenAppliances
Lightning2
Lightning7
Mallat
Meat
MedicalImages
MiddlePhalanxOutlineAgeGroup
MiddlePhalanxOutlineCorrect
MiddlePhalanxTW
MixedShapesRegularTrain
MixedShapesSmallTrain
MoteStrain
NonInvasiveFetalECGThorax1
NonInvasiveFetalECGThorax2
OSULeaf
OliveOil
PhalangesOutlinesCorrect
Phoneme
PigAirwayPressure
PigArtPressure
PigCVP


/tmp/ipykernel_5952/3333193021.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'lr=2.0e-04, k=3, dropout=0.3 blocks=2, ch=[1, 64, 128], layers=2, fc=[512, 256], batch=16, epochs=82' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df["Dataset"] == "AVERAGE", f"{trial_name}_params"] = hp_str
[I 2025-10-07 14:29:07,897] Trial 0 finished with value: 0.6678490115108205 and parameters: {'dropout': 0.3, 'lr': 0.0001976236811357124, 'k_size': 3, 'n_blocks': 2, 'channels_block0': 64, 'channels_block1': 128, 'n_layers': 2, 'layer_size0': 256, 'layer_size1': 256, 'batch_size': 16, 'epochs': 82}. Best is trial 0 with value: 0.6678490115108205.


ACSF1
Adiac
ArrowHead
BME
Beef
BeetleFly
BirdChicken
CBF
Car
Chinatown
ChlorineConcentration
CinCECGTorso
Coffee
Computers
CricketX
CricketY
CricketZ
Crop
DiatomSizeReduction
DistalPhalanxOutlineAgeGroup
DistalPhalanxOutlineCorrect
DistalPhalanxTW
ECG200
ECG5000
ECGFiveDays
EOGHorizontalSignal
EOGVerticalSignal
Earthquakes
ElectricDevices
